In [14]:
!pip install --quiet huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.6 MB/s eta 0:00:00


In [1]:
# from huggingface_hub import notebook_login
# notebook_login()

In [66]:
!pip install --quiet transformers accelerate sentencepiece

from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

print("Model loaded!")


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Model loaded!


In [67]:
!pip install --quiet transformers accelerate sentencepiece faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 10.3 MB/s eta 0:00:00


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL = "google/gemma-2b-it"   # หรือ model ที่คุณมีสิทธิ์ใช้
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

tokenizer = AutoTokenizer.from_pretrained(MODEL, use_auth_token=True)  # ถ้าล็อกอินด้วย notebook_login() ก็ใช้ได้
model = AutoModelForCausalLM.from_pretrained(MODEL, use_auth_token=True)
model.to(device)

# ถ้าโมเดลยังไม่มี pad_token ให้เซ็ต (safety)
if model.config.pad_token_id is None:
    model.config.pad_token_id = model.config.eos_token_id
    tokenizer.pad_token = tokenizer.eos_token

print("Loaded:", MODEL)


Using device: cpu


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1041: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded: google/gemma-2b-it


In [5]:
import math

def generate_text(prompt,
                  max_new_tokens=150,
                  temperature=0.7,
                  top_p=0.9,
                  do_sample=True,
                  eos_token_id=None):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_p=top_p,
        eos_token_id=eos_token_id if eos_token_id is not None else model.config.eos_token_id,
        pad_token_id=model.config.pad_token_id,
        # optional safety tweaks:
        no_repeat_ngram_size=3,   # ลดการซ้ำ
        early_stopping=True
    )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    # ตัด prompt ส่วนต้น (ถ้าต้องการให้ return only generated part)
    generated = text[len(prompt):].strip()
    return text, generated

# ตัวอย่าง
p = "Explain cross validation in simple words:"
# full, gen = generate_text(p, max_new_tokens=150, temperature=0.9)
# full, gen = generate_text(p, max_new_tokens=150, temperature=0.5)
full, gen = generate_text(p, max_new_tokens=150, temperature=0.1)
print("=== Full ===\n", full)
# print("=== Generated only ===\n", gen)


=== Full ===
 Explain cross validation in simple words:

Imagine you have a set of data and you want to know how well your model will perform on unseen data. Cross validation is a technique that allows you to split your data into multiple folds, train your model on each fold, and then test it on the remaining folds. This process is repeated multiple times, each time with a different fold held out as the test set.

By doing this, you can get a more accurate estimate of how your model's performance will be on unseen, unlabeled data. This is especially useful when you have limited data or when you want a model that is robust to outliers.


In [7]:
# simple chat loop (stateless prompt composition)
def chat_with_history(history, user_msg, max_new_tokens=150):
    # history: list of (role, text) e.g. [("system","You are a helpful assistant."), ("user","hi")]
    # build prompt (simple format)
    system_prompt = ""
    for role, text in history:
        if role == "system":
            system_prompt += f"System: {text}\n"
        elif role == "user":
            system_prompt += f"User: {text}\n"
        else:
            system_prompt += f"{role.capitalize()}: {text}\n"
    system_prompt += f"User: {user_msg}\nAssistant:"

    full, gen = generate_text(system_prompt, max_new_tokens=max_new_tokens, temperature=0.7)
    return full, gen

# usage:
history = [("system","You are a helpful tutor who explains simply.")]
full, gen = chat_with_history(history, "What is regularization? Give short bullets.")
# print(gen)
print(full)


System: You are a helpful tutor who explains simply.
User: What is regularization? Give short bullets.
Assistant: Sure! Here's a simple explanation of regularization:

- Regularization is a technique used in machine learning and statistics to prevent overfitting.
- It involves adding a penalty term to the loss function that is proportional to the size of the model's weights.
 - This penalty term encourages the model to find a simpler solution that is less likely to overfit.
 
- Regularized models tend to be simpler and easier to interpret.


In [8]:
prompt_template = """You are a patient tutor. Answer concisely and clearly.

Question: {question}

Constraints:
- Use plain language
- Max 50 words
- Provide 2 short bullet points as actions if applicable

Answer:
"""
q = "How to prevent overfitting in ML?"
print(generate_text(prompt_template.format(question=q), max_new_tokens=120, temperature=0.3)[1])


Sure! Overfitting occurs when a model becomes too specific to the training data. To prevent this, try:

- **Randomize the training process**: Shuffle the data before training.
- **Regularize the model**: Use techniques like L1 or L2 regularization.


In [9]:
def count_tokens(text):
    return len(tokenizer.encode(text))

example_prompt = "Explain cross validation in simple words:"
print("Tokens in prompt:", count_tokens(example_prompt))
print("Max model context (approx):", getattr(model.config, "n_ctx", "unknown"))


Tokens in prompt: 8
Max model context (approx): unknown


In [10]:
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer("all-MiniLM-L6-v2")  # เบา และดี
texts = [
    "Cross validation is a method to evaluate models.",
    "Overfitting means model memorizes training data.",
    "Regularization penalizes complexity."
]
embs = embed_model.encode(texts, convert_to_numpy=True)
print("Emb shape:", embs.shape)
# similarity example (cosine)
import numpy as np
def cos_sim(a,b): return (a@b)/(np.linalg.norm(a)*np.linalg.norm(b))
print("sim 0-1:", cos_sim(embs[0], embs[1]))


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Emb shape: (3, 384)
sim 0-1: 0.3313778


In [12]:
# 1) documents -> chunk
docs = [
    "Machine learning models sometimes overfit when trained too long on small data.",
    "Cross-validation splits data into folds to validate performance.",
    "Regularization, more data, and early stopping help avoid overfitting."
]
# chunking here trivial (docs already small). In real case use text splitter.

# 2) embed chunks
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer("all-MiniLM-L6-v2")
vecs = embedder.encode(docs, convert_to_numpy=True)

# 3) build FAISS index
import faiss
dim = vecs.shape[1]
index = faiss.IndexFlatIP(dim)  # inner product (use normalized for cosine)
faiss.normalize_L2(vecs)
index.add(vecs)

# 4) query
query = "How do we stop overfitting?"
qvec = embedder.encode([query], convert_to_numpy=True)
faiss.normalize_L2(qvec)
D, I = index.search(qvec, k=2)
print("Top chunks indices:", I, "scores:", D)

# 5) build context for LLM
top_chunks = [docs[i] for i in I[0]]
context = "\n\n".join(top_chunks)
prompt = f"Use the following context to answer question.\n\nContext:\n{context}\n\nQuestion: {query}\nAnswer concisely:"
print(prompt)

# 6) ask Gemma
full, gen = generate_text(prompt, max_new_tokens=120, temperature=0.2)
print("Answer:\n", gen)

# นี่คือ pipeline RAG แบบย่อ: chunk → embed → retrieve → prompt with context → generate ที่เรากำลังทดลอง

Top chunks indices: [[2 0]] scores: [[0.66413844 0.6175256 ]]
Use the following context to answer question.

Context:
Regularization, more data, and early stopping help avoid overfitting.

Machine learning models sometimes overfit when trained too long on small data.

Question: How do we stop overfitting?
Answer concisely:
Answer:
 Regularization, early stopping, and data augmentation can help prevent overfitting in machine learning models.


In [13]:
# save small results
with open("rag_example_context.txt","w",encoding="utf-8") as f:
    f.write(context)
print("Saved context")

# warnings:
# - อย่าใส่ข้อมูลลับ (PII) ลงใน public models เก็บไว้ใน secure vector db ถ้าจริงจัง
# - ตรวจผลลัพธ์เสมอ เพราะ LLM อาจ hallucinate


Saved context
